In [ ]:
%load_ext cython
%load_ext line_profiler
from Cython.Compiler.Options import get_directive_defaults

get_directive_defaults()['linetrace'] = True
get_directive_defaults()['binding'] = True

In [1]:
import numpy as np
from pycalphad import CompiledModel, Model, Database, calculate, equilibrium
from pycalphad.core.eqsolver import _solve_eq_at_conditions
import pycalphad.variables as v

#dbf = Database('2016-08-10-AlGdMgand18RLPSO-for 3d plot.tdb')
dbf = Database('alfe_sei.TDB')

In [2]:
import pstats, cProfile

#models = {name: CompiledModel(dbf, ['AL', 'FE', 'VA'], name) for name in dbf.phases.keys()}
#cProfile.runctx("equilibrium(dbf, ['AL', 'FE', 'VA'], dbf.phases.keys(), {v.T: 1000, v.X('AL'): (0,1,0.01), v.P: 101325}, model=models)", globals(), locals(), "Profile.prof")
#s = pstats.Stats("Profile.prof")
#s.strip_dirs().sort_stats("tottime").print_stats()
eq = equilibrium(dbf, ['AL', 'FE', 'VA'], dbf.phases.keys(), {v.T: 1000, v.X('AL'): 0.1, v.P: 101325}, verbose=True)

Calculation Backend: Compiled (autowrap)
Components: AL FE VA
Phases: AL13FE4 AL2FE AL5FE2 AL5FE4 B2_BCC BCC_A2 FCC_A1 HCP_A3 LIQUID [done]
('Composition Sets', [CompositionSet(B2_BCC, [ 0.11111111  0.88888889]), CompositionSet(B2_BCC, [ 0.05555556  0.94444444])])
('NEW_L_MULTIPLIERS', array([  -1317.94916933,   -1317.94916933,     926.86058643,
           839.90874505,    -585.01538944,  -27488.56710902,
       -126136.93871556,  -43581.92654995]))
('old_driving_force', -52310.477710388084)
(1.0, -52093.079553656855, 0.008332613323217497)
(0.5, -52138.059520487157, 0.00061219243322951922)
(0.25, -52208.399001067679, 0.00015304810830729654)
(0.125, -52255.426669399792, 3.8262027076824134e-05)
(0.0625, -52281.937570527996, 9.5655067691990947e-06)
(0.03125, -52295.952030165521, 2.3913766922928348e-06)
(0.015625, -52303.15068807346, 5.9784417308361704e-07)
(0.0078125, -52306.798116369158, 1.4946104327784315e-07)
(0.00390625, -52308.633887846801, 3.7365260815991341e-08)
(0.001953125, -5230

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(eq.coords['X_AL'], eq.GM[0,0])
plt.show()

In [ ]:
res = calculate(dbf, ['AL', 'FE', 'VA'], ['B2_BCC'], T=790, P=101325, model=Model, pdens=50)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(res.X[0,0].sel(component='AL'), res.GM[0,0])
#plt.plot([1,0], [-105391.47499307,  -31879.10489844])

#plt.xlim((0.135, 0.145))
#plt.ylim((-45000, -40000))
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycalphad import equilibrium, calculate
from pycalphad import Database, Model
import pycalphad.variables as v
import numpy as np
from pycalphad.plot.utils import phase_legend

db = Database('alfe_sei.TDB')
phase_names=set(sorted(list(db.phases.keys()))) - {'BCC_A2'}
eq = equilibrium(db, ['AL', 'FE', 'VA'], phase_names, {v.X('AL'): 0.23, v.T: 1170, v.P: 101325}, verbose=True)

print(eq)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycalphad import equilibrium, calculate
from pycalphad import Database, Model
import pycalphad.variables as v
import numpy as np
from pycalphad.plot.utils import phase_legend

db = Database('alfe_sei.TDB')
phase_names=sorted(list(db.phases.keys()))

my_phases = ['LIQUID', 'FCC_A1']
phase_handles, phasemap = phase_legend(my_phases)

Tvector=np.arange(900.0,950.0,1.0)

eq = equilibrium(db, ['AL', 'FE', 'VA'], my_phases, {v.X('FE'): 1.5e-3, v.T: Tvector, v.P: 101325}, verbose=False)

print(eq)

plt.figure()

for name in my_phases:
    phase_indices = np.nonzero(eq.Phase.values == name)
    plt.scatter(np.take(eq['T'].values, phase_indices[1]), eq.NP.values[phase_indices], color=phasemap[name])
    plt.gca().legend(phase_handles, my_phases, loc='lower right')
    plt.xlabel('Temperature (K)')
    plt.ylabel('Phase Fraction')
plt.figure()
plt.scatter(eq['T'].values, eq.MU.sel(component='FE').isel(P=0))
plt.xlabel('Temperature (K)')
plt.ylabel('MU(FE)')

In [ ]:
eq.GM

In [ ]:
import numpy as np
from pycalphad.plot.utils import phase_legend
fig = plt.figure()
phase_handles, phasemap = phase_legend(['LIQUID', 'FCC_A1'])
for phase, color in phasemap.items():
    res = calculate(db, ['AL', 'FE', 'VA'], phase, T=901, P=101325)
    order = np.argsort(res.X[0,0].sel(component='FE'))
    x = res.X[0,0].sel(component='FE').values[order]
    y = res.GM[0,0].values[order]
    fig.gca().plot(x, y, c=color)
fig.gca().vlines(1.5e-3, -37000, -35000, colors='r', linestyles='dotted')
fig.gca().set_xlim((0, 2e-2))
fig.gca().set_ylim((-37000, -35000))
fig.gca().set_xlabel('X(FE)')
fig.gca().set_ylabel('Molar Gibbs Energy (J/mol-atom)')
fig.gca().set_title('T = 901 K')
fig.gca().legend(phase_handles, ['LIQUID', 'FCC_A1'], loc='lower right')

In [ ]:
from pycalphad import equilibrium, calculate
from pycalphad import Database, Model
import pycalphad.variables as v
from sympy import Symbol
dbf = Database('alfe_sei.TDB')
#calculate(dbf, ['AL', 'VA'], 'FCC_A1', T=1200, P=101325)
eq = equilibrium(dbf, ['AL', 'VA'], 'FCC_A1', {v.T: 1200, v.P: 101325}, verbose=True, output='SM', parameters={'GHSERAL': 1000})

In [ ]:
eq